In [1]:
import pandas as pd
from soccerapi.api import Api888Sport
# from soccerapi.api import ApiUnibet
# from soccerapi.api import ApiBet365

api = Api888Sport()
url = 'https://www.888sport.com/#/filter/football/italy/serie_a'
odds = api.odds(url)

predictions_link = 'C:/Users/99451/Desktop/MODEL/2025/2024-10-13.xlsx'

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Use json_normalize to flatten nested dictionaries
df = pd.json_normalize(odds)
competitions = api.competitions()
competitions_df = pd.json_normalize(competitions)

# Transpose the DataFrame
competitions_df_transposed = competitions_df.T

# Reset index to turn the index into a column
competitions_df_transposed.reset_index(inplace=True)

competitions_df_transposed.columns = ['League', 'Links']

# Display the DataFrame
competitions_df_transposed.head()


,League,Links
0,Italy.Serie A,https://www.888sport.com/#/filter/football/ita...
1,Italy.Serie B,https://www.888sport.com/#/filter/football/ita...
2,Italy.Serie C Girone C,https://www.888sport.com/#/filter/football/ita...
3,Italy.Serie C Girone A,https://www.888sport.com/#/filter/football/ita...
4,Italy.Serie C Girone B,https://www.888sport.com/#/filter/football/ita...


In [2]:
matches = pd.DataFrame()

for i in range(len(competitions_df_transposed)):
    url = competitions_df_transposed['Links'].iloc[i]
    odds = api.odds(url)
    my_df = pd.json_normalize(odds)
    matches = pd.concat([matches, my_df], ignore_index = True)

# Automatically select all numeric columns and divide by 1000
numeric_cols = matches.select_dtypes(include='number').columns
matches[numeric_cols] = matches[numeric_cols] / 1000
matches.head()

,time,home_team,away_team,full_time_result.1,full_time_result.X,full_time_result.2,under_over.O2.5,under_over.U2.5,both_teams_to_score.yes,both_teams_to_score.no,double_chance.1X,double_chance.12,double_chance.2X,both_teams_to_score,under_over,double_chance,full_time_result
0,2024-10-19T13:00:00Z,Genoa,Bologna,3.10,3.20,2.38,2.25,1.61,1.90,1.79,1.58,1.35,1.38,NaN,NaN,NaN,NaN
1,2024-10-19T13:00:00Z,Como,Parma,1.93,3.85,3.55,1.67,2.15,1.58,2.18,1.30,1.26,1.84,NaN,NaN,NaN,NaN
2,2024-10-19T16:00:00Z,AC Milan,Udinese,1.35,5.20,8.00,1.57,2.35,1.81,1.89,1.08,1.17,3.05,NaN,NaN,NaN,NaN
3,2024-10-19T18:45:00Z,Juventus,Lazio,1.90,3.40,4.20,2.23,1.63,1.96,1.74,1.24,1.32,1.86,NaN,NaN,NaN,NaN
4,2024-10-20T10:30:00Z,Empoli,Napoli,5.25,4.10,1.61,1.92,1.84,1.91,1.78,2.25,1.24,1.17,NaN,NaN,NaN,NaN


In [3]:
predictions = pd.read_excel(predictions_link)
predictions['2.5U'] = 100 - predictions['2.5O']
predictions['OTTS'] = 100 - predictions['BTTS']

selected_columns = ['League','Home','Away','FT1', 'FTX', 'FT2', '2.5O', '2.5U','BTTS', 'OTTS', 'DC1X', 'DC12', 'DCX2']
predictions = predictions[selected_columns]

print('Number of matches predicted: ', len(predictions))
predictions.head()

Number of matches predicted:  11


,League,Home,Away,FT1,FTX,FT2,2.5O,2.5U,BTTS,OTTS,DC1X,DC12,DCX2
0,Portugal2,Vizela,Oliveirense,58.61,30.99,10.39,33.79,66.21,35.21,64.79,89.60,69.00,41.38
1,Portugal2,Academico Viseu,Maritimo,56.51,23.22,20.19,72.52,27.48,70.36,29.64,79.73,76.70,43.41
2,Portugal2,Pacos Ferreira,Torreense,18.27,26.67,55.03,59.65,40.35,60.30,39.70,44.94,73.30,81.70
3,Portugal2,Chaves,Feirense,13.69,39.17,47.15,20.42,79.58,27.88,72.12,52.86,60.84,86.32
4,Spain2,Huesca,Albacete,75.45,10.27,13.93,77.53,22.47,62.89,37.11,85.72,89.38,24.20


In [4]:
import pandas as pd
from fuzzywuzzy import process

#Renaming matches column to match with predictions
matches = matches.rename(columns={'home_team': 'HomeTeam', 'away_team': 'AwayTeam'})

# Function to match similar strings
def fuzzy_merge(df1, df2, key1, key2, threshold=75, limit=1):
    """
    df1, df2: DataFrames to merge
    key1, key2: Column names on which to perform the fuzzy match
    threshold: Similarity threshold (0-100), the higher the stricter
    limit: Maximum matches to return
    """
    s = df2[key2].tolist()

    # Create a new DataFrame to store the best match
    matches = df1[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))
    
    # Extract matched values and merge
    df1['Best_Match'] = matches.apply(lambda x: x[0] if x else None)
    df1['Match_Score'] = matches.apply(lambda x: x[1] if x else None)
    
    # Merge on the best match
    return pd.merge(df1, df2, left_on='Best_Match', right_on=key2)

# Use the fuzzy merge function
result = fuzzy_merge(predictions, matches, 'Home', 'HomeTeam')
result.drop(['both_teams_to_score', 'under_over', 'double_chance', 'full_time_result'], axis=1, inplace = True)
print('Number of matches found with odds: {} number of matches missing: {}'.format(len(result), len(predictions) - len(result)))
result.head()


Number of matches found with odds: 10 number of matches missing: 1


,League,Home,Away,FT1,FTX,FT2,2.5O,2.5U,BTTS,OTTS,DC1X,DC12,DCX2,Best_Match,Match_Score,time,HomeTeam,AwayTeam,full_time_result.1,full_time_result.X,full_time_result.2,under_over.O2.5,under_over.U2.5,both_teams_to_score.yes,both_teams_to_score.no,double_chance.1X,double_chance.12,double_chance.2X
0,Portugal2,Vizela,Oliveirense,58.61,30.99,10.39,33.79,66.21,35.21,64.79,89.60,69.00,41.38,Vizela,100.0,2024-10-13T10:00:00Z,Vizela,UD Oliveirense,1.65,3.70,4.50,1.73,1.92,1.74,2.00,1.17,1.23,2.07
1,Portugal2,Academico Viseu,Maritimo,56.51,23.22,20.19,72.52,27.48,70.36,29.64,79.73,76.70,43.41,Académico Viseu,97.0,2024-10-13T14:30:00Z,Académico Viseu,Marítimo Funchal,2.43,2.95,2.95,2.23,1.54,1.93,1.80,1.35,1.35,1.50
2,Portugal2,Pacos Ferreira,Torreense,18.27,26.67,55.03,59.65,40.35,60.30,39.70,44.94,73.30,81.70,Paços Ferreira,96.0,2024-10-13T17:00:00Z,Paços Ferreira,Torreense,2.38,3.05,2.95,2.08,1.62,1.84,1.89,1.35,1.33,1.52
3,Portugal2,Chaves,Feirense,13.69,39.17,47.15,20.42,79.58,27.88,72.12,52.86,60.84,86.32,Chaves,100.0,2024-10-13T19:30:00Z,Chaves,Feirense,1.84,3.30,4.10,1.98,1.68,1.87,1.86,1.19,1.28,1.88
4,Spain2,Huesca,Albacete,75.45,10.27,13.93,77.53,22.47,62.89,37.11,85.72,89.38,24.20,Huesca,100.0,2024-10-13T12:00:00Z,Huesca,Albacete Balompié,2.23,2.85,3.45,2.28,1.56,1.90,1.83,1.26,1.37,1.58


# Calculating probabilities and finding value bets

In [5]:
ftrs = []
ftrs_string = []
ftrs_percent = []
ftrs_odds = []
overs = []
overs_string = []
overs_percent = []
overs_odds = []
btts = []
btts_string = []
btts_percent = []
btts_odds = []
dcs = []
dcs_string = []
dcs_percent = []
dcs_odds = []

for i in range(len(result)):
    if result['DC1X'].iloc[i] > result['DC12'].iloc[i] and result['DC1X'].iloc[i] > result['DCX2'].iloc[i]:
        dcs_string.append('DC1X')
        dcs_odds.append(result['double_chance.1X'].iloc[i])
        dcs_percent.append(result['DC1X'].iloc[i])
        value = result['double_chance.1X'].iloc[i] - (1 / (result['DC1X'].iloc[i] / 100))
        dcs.append(round(value,2))
    elif result['DCX2'].iloc[i] > result['DC1X'].iloc[i] and result['DCX2'].iloc[i] > result['DC12'].iloc[i]:
        dcs_string.append('DCX2')
        dcs_odds.append(result['double_chance.2X'].iloc[i])
        dcs_percent.append(result['DCX2'].iloc[i])
        value = result['double_chance.2X'].iloc[i] - (1 / (result['DCX2'].iloc[i] / 100))
        dcs.append(round(value,2))
    else:
        dcs_string.append('DC12')
        dcs_odds.append(result['double_chance.12'].iloc[i])
        dcs_percent.append(result['DC12'].iloc[i])
        value = result['double_chance.12'].iloc[i] - (1 / (result['DC12'].iloc[i] / 100))
        dcs.append(round(value,2))

    if result['FT1'].iloc[i] > result['FTX'].iloc[i] and result['FT1'].iloc[i] > result['FT2'].iloc[i]:
        ftrs_string.append('FT1')
        ftrs_odds.append(result['full_time_result.1'].iloc[i])
        ftrs_percent.append(result['FT1'].iloc[i])
        value = result['full_time_result.1'].iloc[i] - (1 / (result['FT1'].iloc[i] / 100))
        ftrs.append(round(value,2))
    elif result['FTX'].iloc[i] > result['FT1'].iloc[i] and result['FTX'].iloc[i] > result['FT2'].iloc[i]:
        ftrs_string.append('FTX')
        ftrs_odds.append(result['full_time_result.X'].iloc[i])
        ftrs_percent.append(result['FTX'].iloc[i])
        value = result['full_time_result.X'].iloc[i] - (1 / (result['FTX'].iloc[i] / 100))
        ftrs.append(round(value,2))
    else:
        ftrs_string.append('FT2')
        ftrs_odds.append(result['full_time_result.2'].iloc[i])
        ftrs_percent.append(result['FT2'].iloc[i])
        value = result['full_time_result.2'].iloc[i] - (1 / (result['FT2'].iloc[i] / 100))
        ftrs.append(round(value,2))
    
    if result['2.5O'].iloc[i] > result['2.5U'].iloc[i]:
        overs_string.append('2.5O')
        overs_odds.append(result['under_over.O2.5'].iloc[i])
        overs_percent.append(result['2.5O'].iloc[i])
        value = result['under_over.O2.5'].iloc[i] - (1 / (result['2.5O'].iloc[i] / 100))
        overs.append(round(value, 2))
    else:
        overs_string.append('2.5U')
        overs_odds.append(result['under_over.U2.5'].iloc[i])
        overs_percent.append(result['2.5U'].iloc[i])
        value = result['under_over.U2.5'].iloc[i] - (1 / (result['2.5U'].iloc[i] / 100))
        overs.append(round(value, 2))

    if result['BTTS'].iloc[i] > result['OTTS'].iloc[i]:
        btts_string.append('BTTS')
        btts_odds.append(result['both_teams_to_score.yes'].iloc[i])
        btts_percent.append(result['BTTS'].iloc[i])
        value = result['both_teams_to_score.yes'].iloc[i] - (1 / (result['BTTS'].iloc[i] / 100))
        btts.append(round(value, 2))
    else:
        btts_string.append('OTTS')
        btts_odds.append(result['both_teams_to_score.no'].iloc[i])
        btts_percent.append(result['OTTS'].iloc[i])
        value = result['both_teams_to_score.no'].iloc[i] - (1 / (result['OTTS'].iloc[i] / 100))
        btts.append(round(value, 2))

    
# Create a DataFrame
df = pd.DataFrame({
    'League': result['League'],
    'HomeTeam': result['HomeTeam'],
    'AwayTeam': result['AwayTeam'],
    'DCBets': dcs_string,
    'DCPercent': dcs_percent,
    'DCOdd': dcs_odds,
    'DCVal': dcs,
    'FTRBets': ftrs_string,
    'FTRPercent': ftrs_percent,
    'FTROdd': ftrs_odds,
    'FTRVal': ftrs,
    '2.5O/U': overs_string,
    '2.5Percent': overs_percent,
    '2.5Odd': overs_odds,
    '2.5Val': overs,
    'BTTS': btts_string,
    'BTPercent': btts_percent,
    'BTOdd': btts_odds,
    'BTVal': btts
})

#Adding recommended bets according to value difference
rec_bet, rec_per, rec_odd, rec_val = [], [], [], []

for i in range(len(df)):
    values = [df['DCVal'].iloc[i], df['FTRVal'].iloc[i], df['2.5Val'].iloc[i], df['BTVal'].iloc[i]]
    if max(values) == df['DCVal'].iloc[i]:
        rec_bet.append(df['DCBets'].iloc[i])
        rec_per.append(df['DCPercent'].iloc[i])
        rec_odd.append(df['DCOdd'].iloc[i])
        rec_val.append(df['DCVal'].iloc[i])
    elif max(values) == df['FTRVal'].iloc[i]:
        rec_bet.append(df['FTRBets'].iloc[i])
        rec_per.append(df['FTRPercent'].iloc[i])
        rec_odd.append(df['FTROdd'].iloc[i])
        rec_val.append(df['FTRVal'].iloc[i])
    elif max(values) == df['2.5Val'].iloc[i]:
        rec_bet.append(df['2.5O/U'].iloc[i])
        rec_per.append(df['2.5Percent'].iloc[i])
        rec_odd.append(df['2.5Odd'].iloc[i])
        rec_val.append(df['2.5Val'].iloc[i])
    else:
        rec_bet.append(df['BTTS'].iloc[i])
        rec_per.append(df['BTPercent'].iloc[i])
        rec_odd.append(df['BTOdd'].iloc[i])
        rec_val.append(df['BTVal'].iloc[i])

rec_df = pd.DataFrame({
    'Rec': rec_bet,
    'RecPer': rec_per,
    'RecOdd': rec_odd,
    'RecVal': rec_val
})

# Concatenate the new DataFrame with the existing one
df = pd.concat([rec_df, df], axis=1)

first_three_columns = ['League', 'HomeTeam', 'AwayTeam']
new_columns = ['Rec', 'RecPer', 'RecOdd', 'RecVal']
remaining_columns = [col for col in df.columns if col not in (first_three_columns + new_columns)]
# Rearrange the columns
df = df[first_three_columns + new_columns + remaining_columns]

# Function to highlight values higher than threshold
def highlight_values(value):
    if isinstance(value, str) or pd.isna(value):
        return ''  # Return empty string for NaN values
    elif value > 80:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = df.style.applymap(highlight_values)
styled_df.to_excel('final_games.xlsx', index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

C:\Users\99451\AppData\Local\Temp\ipykernel_15816\1761804651.py:161: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(highlight_values)


,League,HomeTeam,AwayTeam,Rec,RecPer,RecOdd,RecVal,DCBets,DCPercent,DCOdd,DCVal,FTRBets,FTRPercent,FTROdd,FTRVal,2.5O/U,2.5Percent,2.5Odd,2.5Val,BTTS,BTPercent,BTOdd,BTVal
0,Portugal2,Vizela,UD Oliveirense,OTTS,64.790000,2.000000,0.460000,DC1X,89.600000,1.170000,0.050000,FT1,58.610000,1.650000,-0.060000,2.5U,66.210000,1.920000,0.410000,OTTS,64.790000,2.000000,0.460000
1,Portugal2,Académico Viseu,Marítimo Funchal,2.5O,72.520000,2.230000,0.850000,DC1X,79.730000,1.350000,0.100000,FT1,56.510000,2.430000,0.660000,2.5O,72.520000,2.230000,0.850000,BTTS,70.360000,1.930000,0.510000
2,Portugal2,Paços Ferreira,Torreense,FT2,55.030000,2.950000,1.130000,DCX2,81.700000,1.520000,0.300000,FT2,55.030000,2.950000,1.130000,2.5O,59.650000,2.080000,0.400000,BTTS,60.300000,1.840000,0.180000
3,Portugal2,Chaves,Feirense,FT2,47.150000,4.100000,1.980000,DCX2,86.320000,1.880000,0.720000,FT2,47.150000,4.100000,1.980000,2.5U,79.580000,1.680000,0.420000,OTTS,72.120000,1.860000,0.470000
4,Spain2,Huesca,Albacete Balompié,2.5O,77.530000,2.280000,0.990000,DC12,89.380000,1.370000,0.250000,FT1,75.450000,2.230000,0.900000,2.5O,77.530000,2.280000,0.990000,BTTS,62.890000,1.900000,0.310000
5,Spain2,Racing Santander,Levante,DC12,84.090000,1.260000,0.070000,DC12,84.090000,1.260000,0.070000,FT1,42.500000,1.900000,-0.450000,2.5O,62.060000,1.580000,-0.030000,BTTS,59.980000,1.500000,-0.170000
6,Spain2,Real Oviedo,Almeria,FT2,43.340000,3.150000,0.840000,DC12,79.380000,1.320000,0.060000,FT2,43.340000,3.150000,0.840000,2.5U,66.440000,1.710000,0.200000,OTTS,64.020000,1.960000,0.400000
7,Spain2,Burgos,Mirandés,2.5U,94.980000,2.120000,1.070000,DCX2,75.320000,1.580000,0.250000,FTX,46.460000,2.800000,0.650000,2.5U,94.980000,2.120000,1.070000,OTTS,91.420000,1.490000,0.400000
8,Spain2,Granada,Córdoba CF,2.5O,71.580000,1.950000,0.550000,DC12,84.510000,1.300000,0.120000,FT1,44.810000,2.080000,-0.150000,2.5O,71.580000,1.950000,0.550000,BTTS,68.030000,1.760000,0.290000
9,Spain2,Elche,Deportivo La Coruña,OTTS,64.930000,1.870000,0.330000,DC12,80.150000,1.350000,0.100000,FT1,51.930000,1.980000,0.050000,2.5U,65.720000,1.620000,0.100000,OTTS,64.930000,1.870000,0.330000
